In [ ]:
from multiprocessing import Process, JoinableQueue as Queue
from os import listdir, makedirs, path

import re
import subprocess

INPUT_PATH = "/home/vmubuntu/OitoDeJaneiro/vids/FULL-0801"
OUTPUT_PATH = "/home/vmubuntu/OitoDeJaneiro/vids/0801-500"

INPUT_DIRS = [
  "01-COBERTURA-PP-OESTE",
  "02-COBERTURA-ENTRADA-PR",
  "03-ENTRADA-PP-RAIO-X-TERREO",
  "04-PALACIO-ALA-LESTE-TERREO",
  "05-PALACIO-ALA-OESTE",
  "06-PRIVATIVO-COMBOIO",
  "07-SALA-PG-3-PISO",
  "08-SUBSOLO-ADM",
  "09-ELEVADOR-DE-SERVICO-TERREO",
  "10-SALAO-NOBRE",
  "11-SALAO-OESTE",
  "12-2o-ANDAR-RAMPA",
  "13-2o-ANDAR-ELEVADOR-LESTE",
  "14-2o-ANDAR-ELEVADOR-OESTE",
  "15-MESANINO",
  "16-CAMERA-VIP-PR",
  "17-3o-ANDAR-SALA-PR",
  "18-3o-ANDAR-SALA-DE-AUDIENCIA",
  "19-3o-ANDAR-CORREDOR-ACESSO-LESTE",
  "20-ELEVADORES-ALA-OESTE",
  "21-2o-ANDAR-ESCADA-OESTE",
  "22-3o-ANDAR-ESCADA-OESTE",
  "23-ELEVADOR-MINISTROS",
  "24-4o-ANDAR-ELEVADOR-LESTE",
  "25-4o-ANDAR-ESCADA-OESTE",
  "26-4o-ANDAR-ELEVADOR-OESTE",
  "27-4o-ANDAR-LADO-LESTE",
  "28-4o-ANDAR-LADO-OESTE",
  "29-ANEXO-I",
  "30-ANEXO-I-ENTRADA-PRINCIPAL-CATRACAS",
  "31-ANEXO-I-CONCHA-ANEXO",
  "32-ANEXO-III-REFEITORIO-CREDEN",
  "33-ANEXO-III-REFEITORIO",
]

FFMPEG_CMD = 'ffmpeg -i "INPUT_PATH" -vf scale=500:-2 -an "OUTPUT_PATH"'
MV_CMD = 'mv "INPUT_PATH" "OUTPUT_PATH"'

In [ ]:
def clean_name(inp):
  out_name = inp.replace(" ", "-").replace("Ã", "A")
  out_name = out_name.replace("(", "-").replace(")", "-")
  out_name = out_name.replace("-.", ".")
  out_name = re.sub(r'[-]+', '-', out_name)
  out_name = re.sub(r'[_]+', '_', out_name)
  out_name = re.sub(r'^([234])-ANDAR', r'\1o-ANDAR', out_name)
  out_name = re.sub(r'_08012023([0-9]{6})', r'_08012023_\1', out_name)
  return out_name

### RESIZE VIDEOS

In [ ]:
def run_ffmpeg(q):
  while True:
    try:
      ip, op = q.get(timeout=3)
    except queue.Empty:
      return
    my_cmd = FFMPEG_CMD.replace("INPUT_PATH", ip).replace("OUTPUT_PATH", op)
    print(my_cmd)
    ret = subprocess.run(my_cmd, shell=True)
    q.task_done()

In [ ]:
to_process = Queue()

for io_dir in INPUT_DIRS:
  input_dir_path = path.join(INPUT_PATH, io_dir)
  output_dir_path = path.join(OUTPUT_PATH, io_dir)
  makedirs(output_dir_path, exist_ok=True)

  input_files = sorted([f for f in listdir(input_dir_path) if f.endswith("mp4")])
  for io_file in input_files:
    output_file_path = path.join(output_dir_path, io_file)
    if not path.exists(output_file_path):
      input_file_path = path.join(input_dir_path, io_file)
      to_process.put((input_file_path, output_file_path))

In [ ]:
for _ in range(4):
  Process(target=run_ffmpeg, args=(to_process,)).start()

to_process.join()
print("DONE")

### RENAME DIRS AND FILES

In [ ]:
to_process = Queue()

for input_dir in INPUT_DIRS:
  input_dir_path = path.join(INPUT_PATH, input_dir)
  input_files = sorted([f for f in listdir(input_dir_path) if f.endswith("mp4")])
  for input_file in input_files:
    output_file = clean_name(input_file)
    output_file_path = path.join(input_dir_path, output_file)
    if not path.exists(output_file_path):
      input_file_path = path.join(input_dir_path, input_file)
      to_process.put((input_file_path, output_file_path))

In [ ]:
while not to_process.empty():
    inp, outp = to_process.get()
    my_cmd = MV_CMD.replace("INPUT_PATH", inp).replace("OUTPUT_PATH", outp)
    print(my_cmd)
    ret = subprocess.run(my_cmd, shell=True)